In [1]:
!pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.0 MB/s eta 0:00:00


In [42]:
%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px

@st.cache_data
def load_data():
    df = pd.read_csv("Districtwise_Crime_of_India_2001_to_2014.csv")
    df.columns = df.columns.str.strip().str.upper().str.replace(" ", "_")
    return df

df = load_data()

st.sidebar.header("Filter the data")
years = sorted(df["YEAR"].dropna().unique())

# Exclude A&N ISLANDS and D&N ISLANDS
states = sorted(df["STATE/UT"].dropna().unique())
states = [state for state in states if state not in ["A&N ISLANDS", "D&N ISLANDS"]]

selected_year = st.sidebar.selectbox("Select Year", years)
selected_state = st.sidebar.selectbox("Select State", states)

filtered_df = df[df["YEAR"] == selected_year]
filtered_df = filtered_df[filtered_df["STATE/UT"] == selected_state]

districts = sorted(filtered_df["DISTRICT"].dropna().unique())
selected_district = st.sidebar.selectbox("Select District", districts)

filtered_df = filtered_df[filtered_df["DISTRICT"] == selected_district]

st.title("Crime Dashboard - India")
st.write(f"### Crime statistics for {selected_district}, {selected_state} in {selected_year}")
st.dataframe(filtered_df)

# Cluster similar crime categories
grouped_data = {
    "Theft": filtered_df[["THEFT", "AUTO_THEFT", "OTHER_THEFT"]].sum(axis=1).values[0],
    "Kidnapping & Abduction": filtered_df[[
        "KIDNAPPING_&_ABDUCTION",
        "KIDNAPPING_AND_ABDUCTION_OF_WOMEN_AND_GIRLS",
        "KIDNAPPING_AND_ABDUCTION_OF_OTHERS"
    ]].sum(axis=1).values[0],
    "Dacoity": filtered_df[[
        "DACOITY",
        "PREPARATION_AND_ASSEMBLY_FOR_DACOITY"
    ]].sum(axis=1).values[0],
    "Rape": filtered_df[["RAPE", "OTHER_RAPE"]].sum(axis=1).values[0]
}


# Add the remaining individual crime types (excluding already grouped ones)
excluded = {
    "THEFT", "AUTO_THEFT", "OTHER_THEFT",
    "KIDNAPPING_&_ABDUCTION", "KIDNAPPING_AND_ABDUCTION_OF_WOMEN_AND_GIRLS", "KIDNAPPING_AND_ABDUCTION_OF_OTHERS",
    "DACOITY", "PREPARATION_AND_ASSEMBLY_FOR_DACOITY", "RAPE", "OTHER_RAPE"
}

for col in filtered_df.columns[3:]:
    if col not in excluded:
        grouped_data[col.replace("_", " ").title()] = filtered_df[col].values[0]

# Create DataFrame for plotting
crime_data = pd.DataFrame(grouped_data.items(), columns=["Crime_Type", "Count"])
crime_data = crime_data[crime_data["Count"] > 0]  # Optional: filter out zero values

# Plot the horizontal bar chart
fig = px.bar(
    crime_data.sort_values("Count"),
    y="Crime_Type",
    x="Count",
    title="Crime Counts by Type (Grouped)",
    text="Count",
    color="Count",
    color_continuous_scale="Blues"
)

fig.update_layout(
    height=800,
    width=1000,
    title_x=0.5,
    xaxis_title="Crime Count",
    yaxis_title="Crime Type",
    yaxis_tickangle=0,
)

fig.update_traces(marker_line_color='black', marker_line_width=1)

st.plotly_chart(fig)


Overwriting app.py


In [12]:
from pyngrok import ngrok
import time

# Kill any existing tunnel
ngrok.kill()

# Run Streamlit app in the background
!streamlit run app.py &> /dev/null &

# Wait a few seconds to make sure Streamlit starts
time.sleep(5)

# Open the tunnel with proper parameters
public_url = ngrok.connect(addr="8501", proto="http")
print("🌐 Streamlit app is live at:", public_url)

🌐 Streamlit app is live at: NgrokTunnel: "https://29a5-35-194-254-26.ngrok-free.app" -> "http://localhost:8501"


In [10]:
!ngrok config add-authtoken 2vdfKp60gecnjpSX8QshzTEXQIh_5RRSLC1nEmgbxFCwGEqdm

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [34]:
import pandas as pd
df = pd.read_csv("Districtwise_Crime_of_India_2001_to_2014.csv")
df.columns

Index(['Unnamed: 0', 'STATE/UT', 'DISTRICT', 'YEAR', 'MURDER',
       'ATTEMPT TO MURDER', 'CULPABLE HOMICIDE NOT AMOUNTING TO MURDER',
       'RAPE', 'CUSTODIAL RAPE', 'OTHER RAPE', 'KIDNAPPING & ABDUCTION',
       'KIDNAPPING AND ABDUCTION OF WOMEN AND GIRLS',
       'KIDNAPPING AND ABDUCTION OF OTHERS', 'DACOITY',
       'PREPARATION AND ASSEMBLY FOR DACOITY', 'ROBBERY', 'BURGLARY', 'THEFT',
       'AUTO THEFT', 'OTHER THEFT', 'RIOTS', 'CRIMINAL BREACH OF TRUST',
       'CHEATING', 'COUNTERFIETING', 'ARSON', 'HURT/GREVIOUS HURT',
       'DOWRY DEATHS', 'ASSAULT ON WOMEN WITH INTENT TO OUTRAGE HER MODESTY',
       'INSULT TO MODESTY OF WOMEN', 'CRUELTY BY HUSBAND OR HIS RELATIVES',
       'IMPORTATION OF GIRLS FROM FOREIGN COUNTRIES',
       'CAUSING DEATH BY NEGLIGENCE', 'OTHER IPC CRIMES', 'TOTAL IPC CRIMES'],
      dtype='object')